In [1]:
# ============================================================================
# SETUP - Run this cell first!
# ============================================================================

import sys
import os
import json
import glob
import random
from datetime import datetime
from pathlib import Path

import numpy as np

# Jupyter async compatibility
import nest_asyncio
nest_asyncio.apply()

# Try uvloop for performance
try:
    import uvloop
    uvloop.install()
    print("✓ uvloop enabled")
except ImportError:
    print("ℹ uvloop not available")

# Add src to path
sys.path.insert(0, str(Path.cwd()))

# Import our solver
from src.run import solve_task, evaluate_task, run_tasks
from src.models import ArcEvaluator
from src.config import MAX_WORKERS, SOLVER_MODELS

print("✓ Imports loaded")
print(f"✓ Max workers: {MAX_WORKERS}")
print(f"✓ Solver models: {[m['id'] for m in SOLVER_MODELS]}")


✓ uvloop enabled
✓ Imports loaded
✓ Max workers: 100
✓ Solver models: ['gpt-5.2', 'gemini-pro', 'gemini-flash']


In [ ]:


from src.perception.analyzer import analyze_task, analyze_grid, analyze_example

# 👇 CHANGE THIS PATH TO ANALYZE ANY TASK
ANALYZE_TASK_PATH = "../ARC-AGI-2/data/evaluation/20a9e565.json"

def print_task_analysis(task_path: str):
    """Print comprehensive task analysis."""
    with open(task_path) as f:
        task_data = json.load(f)
    
    task_id = os.path.basename(task_path).replace('.json', '')
    analysis = analyze_task(task_data, task_id)
    
    print("=" * 70)
    print(f"📋 Task Details")
    print(f"   Task ID: {task_id}")
    print(f"   Dataset: ARC-AGI-2 Evaluation")
    print("=" * 70)
    
    for ex in analysis.train_examples:
        inp = ex.input_analysis
        out = ex.output_analysis
        trans = ex.transform_analysis
        
        print(f"\n{'─'*70}")
        print(f"📚 Training Example #{ex.example_num}")
        print(f"{'─'*70}")
        
        print(f"\n🔹 INPUT")
        print(f"   Grid Size:      {inp.rows} × {inp.cols} ({inp.total_cells} cells)")
        print(f"   Colors Used:    {inp.colors_used} colors")
        print(f"   Color Palette:  {', '.join(inp.color_palette)}")
        print(f"   Background:     {inp.background_color}")
        print(f"   Fill Ratio:     {inp.fill_ratio:.1f}%")
        print(f"   Shapes by Color:")
        for color, count in inp.shapes_by_color.items():
            print(f"      {color}: {count}")
        
        print(f"\n🔹 OUTPUT")
        print(f"   Grid Size:      {out.rows} × {out.cols} ({out.total_cells} cells)")
        print(f"   Colors Used:    {out.colors_used} colors")
        print(f"   Color Palette:  {', '.join(out.color_palette)}")
        print(f"   Background:     {out.background_color}")
        print(f"   Fill Ratio:     {out.fill_ratio:.1f}%")
        print(f"   Shapes by Color:")
        for color, count in out.shapes_by_color.items():
            print(f"      {color}: {count}")
        
        print(f"\n🔄 KEY TRANSFORMATIONS")
        print(f"   Size Change:         {inp.total_cells} cells → {out.total_cells} cells ({trans.size_change_percent:+.1f}%)")
        print(f"   New Colors Intro:    {'YES' if trans.new_colors_introduced else 'NO'}")
        print(f"   Density Change:      {inp.fill_ratio:.1f}% → {out.fill_ratio:.1f}% ({trans.density_change_percent:+.1f}%)")
        print(f"   Shape Count:         {trans.input_shape_count} shapes → {trans.output_shape_count} shapes")
        print(f"   Transform Hints:     {', '.join(trans.hints)}")
    
    # Test inputs (we predict outputs for these - outputs are NOT shown to solver)
    for i, test in enumerate(analysis.test_inputs, 1):
        print(f"\n{'─'*70}")
        label = f"🧪 TEST INPUT #{i}" if len(analysis.test_inputs) > 1 else "🧪 TEST INPUT"
        print(f"{label} (predict output for this)")
        print(f"{'─'*70}")
        
        print(f"\n🔹 INPUT")
        print(f"   Grid Size:      {test.rows} × {test.cols} ({test.total_cells} cells)")
        print(f"   Colors Used:    {test.colors_used} colors")
        print(f"   Color Palette:  {', '.join(test.color_palette)}")
        print(f"   Background:     {test.background_color}")
        print(f"   Fill Ratio:     {test.fill_ratio:.1f}%")
        print(f"   Shapes by Color:")
        for color, count in test.shapes_by_color.items():
            print(f"      {color}: {count}")
    
    # Cross-example patterns
    print(f"\n{'='*70}")
    print(f"🔍 CROSS-EXAMPLE PATTERNS")
    print(f"{'='*70}")
    print(f"   Size always preserved:   {analysis.consistent_size_preservation}")
    print(f"   Colors always preserved: {analysis.consistent_color_preservation}")
    print(f"   Shape count preserved:   {analysis.consistent_shape_count}")
    if analysis.common_hints:
        print(f"   Common hints:            {', '.join(analysis.common_hints)}")

# Run it!
print_task_analysis(ANALYZE_TASK_PATH)


In [ ]:
# ============================================================================
# RUN A SINGLE TASK BY PATH
# ============================================================================

import asyncio

# 👇 CHANGE THIS PATH TO RUN ANY TASK
TASK_PATH = "../ARC-AGI-2/data/evaluation/581f7754.json"

async def run_single():
    # Load task
    with open(TASK_PATH) as f:
        task_data = json.load(f)
    
    task_name = os.path.basename(TASK_PATH)
    n_tests = len(task_data['test'])
    
    # Check for ground truths (ALL test cases)
    has_ground_truth = 'output' in task_data['test'][0]
    ground_truths = [np.array(t['output']) for t in task_data['test']] if has_ground_truth else []
    
    # Prepare for solver - pass ALL test inputs (without outputs)
    task_for_solver = {
        'train': task_data['train'],
        'test': [{'input': t['input']} for t in task_data['test']],
    }
    
    print("=" * 60)
    print(f"📋 Task: {task_name}")
    print(f"   Training examples: {len(task_data['train'])}")
    print(f"   Test inputs: {n_tests}")
    print(f"   Ground truth: {'✓ Available' if has_ground_truth else '✗ Not available'}")
    print("=" * 60)
    
    start_time = datetime.now()
    
    # Solve! - now returns predictions_per_test[test_idx][attempt_idx]
    predictions_per_test, info = await solve_task(task_for_solver, verbose=True)
    
    duration = (datetime.now() - start_time).total_seconds()
    
    # Evaluate using official ARC scoring
    if ground_truths:
        n_correct = 0
        
        print("\n" + "=" * 60)
        print("📊 EVALUATION (Official ARC Scoring)")
        print("=" * 60)
        
        for test_idx, gt in enumerate(ground_truths):
            attempts = predictions_per_test[test_idx]
            match1 = np.array_equal(np.array(attempts[0]), gt)
            match2 = np.array_equal(np.array(attempts[1]), gt) if len(attempts) > 1 else False
            correct = match1 or match2
            
            if correct:
                n_correct += 1
            
            if n_tests > 1:
                print(f"\n   Test {test_idx + 1}/{n_tests}:")
                print(f"      Attempt 1: {'✅ CORRECT' if match1 else '❌ Wrong'}")
                print(f"      Attempt 2: {'✅ CORRECT' if match2 else '❌ Wrong'}")
                print(f"      Result: {'✓ PASS' if correct else '✗ FAIL'}")
            else:
                print(f"   Attempt 1: {'✅ CORRECT' if match1 else '❌ Wrong'}")
                print(f"   Attempt 2: {'✅ CORRECT' if match2 else '❌ Wrong'}")
        
        score = n_correct / n_tests
        print(f"\n   {'─' * 40}")
        print(f"   📊 Task Score: {n_correct}/{n_tests} ({score*100:.0f}%)")
        print(f"   {'🎉 TASK SOLVED!' if score == 1.0 else '❌ Task not fully solved'}")
        print(f"   ⏱️  Time: {duration:.1f}s")
    else:
        print(f"\n⏱️  Completed in {duration:.1f}s (no ground truth)")
    
    return predictions_per_test, info

# Run it!
predictions_per_test, info = asyncio.get_event_loop().run_until_complete(run_single())


In [ ]:
# ============================================================================
# RUN N RANDOM TASKS FROM EVALUATION
# ============================================================================

import asyncio

# 👇 CONFIG
NUM_TASKS = 5  # Number of random tasks to run
EVAL_DIR = "../ARC-AGI-2/data/evaluation"

async def run_random_eval():
    # Get all tasks
    all_tasks = sorted(glob.glob(os.path.join(EVAL_DIR, "*.json")))
    
    # Random sample
    random.seed()  # Use current time for randomness
    selected = random.sample(all_tasks, min(NUM_TASKS, len(all_tasks)))
    
    print("=" * 60)
    print(f"🎲 RANDOM EVALUATION ({len(selected)} tasks)")
    print(f"   Available: {len(all_tasks)} tasks")
    print("=" * 60)
    print(f"\n📋 Selected: {[os.path.basename(p) for p in selected[:5]]}...")
    
    start_time = datetime.now()
    
    # Run tasks
    results = await run_tasks(selected, verbose=True)
    
    duration = (datetime.now() - start_time).total_seconds()
    
    # Summary using official ARC scoring
    total_tests = sum(r.n_test_cases for r in results)
    total_correct = sum(r.n_correct for r in results)
    tasks_fully_solved = sum(1 for r in results if r.score == 1.0)
    errors = sum(1 for r in results if r.error)
    avg_score = sum(r.score for r in results) / len(results) if results else 0
    
    print("\n" + "=" * 60)
    print("🏆 RESULTS (Official ARC Scoring)")
    print("=" * 60)
    print(f"📋 Tasks evaluated: {len(results)}")
    print(f"📋 Total test cases: {total_tests}")
    print(f"✅ Test cases correct: {total_correct}/{total_tests} ({100*total_correct/total_tests:.1f}%)")
    print(f"🎯 Tasks fully solved: {tasks_fully_solved}/{len(results)}")
    print(f"📊 Average task score: {avg_score*100:.1f}%")
    print(f"❌ Errors: {errors}")
    print(f"⏱️  Total time: {duration:.0f}s ({duration/len(results):.1f}s per task avg)")
    
    print("\n📋 Individual results:")
    for r in results:
        status = "✓" if r.score == 1.0 else "◐" if r.score > 0 else "✗"
        tests_info = f" ({r.n_correct}/{r.n_test_cases} tests)" if r.n_test_cases > 1 else ""
        print(f"   {status} {r.task_id}: {r.score*100:.0f}%{tests_info}")
    
    return results

# Run it!
results = asyncio.get_event_loop().run_until_complete(run_random_eval())


In [2]:
# ============================================================================
# RUN N RANDOM TASKS FROM FAILED LIST
# ============================================================================

import asyncio

# 👇 CONFIG
NUM_TASKS = 5  # Number of failed tasks to retry
FAIL_TASKS_FILE = "../fail_tasks.txt"
EVAL_DIR = "../ARC-AGI-2/data/evaluation"

async def run_failed_tasks():
    # Read failed tasks from file
    with open(FAIL_TASKS_FILE, 'r') as f:
        failed_ids = [line.strip() for line in f if line.strip()]
    
    # Build paths (add .json if needed)
    all_failed = []
    for task_id in failed_ids:
        if not task_id.endswith('.json'):
            task_id = task_id + '.json'
        path = os.path.join(EVAL_DIR, task_id)
        if os.path.exists(path):
            all_failed.append(path)
    
    print(f"✓ Found {len(all_failed)}/{len(failed_ids)} valid tasks in fail_tasks.txt")
    
    if not all_failed:
        print("❌ No valid failed tasks found!")
        return []
    
    # Random sample
    random.seed()  # Use current time for randomness
    selected = random.sample(all_failed, min(NUM_TASKS, len(all_failed)))
    
    print("=" * 60)
    print(f"🔄 RETRYING FAILED TASKS ({len(selected)} tasks)")
    print(f"   Total failed: {len(all_failed)} tasks")
    print("=" * 60)
    print(f"\n📋 Selected: {[os.path.basename(p) for p in selected[:5]]}...")
    
    start_time = datetime.now()
    
    # Run tasks
    results = await run_tasks(selected, verbose=True)
    
    duration = (datetime.now() - start_time).total_seconds()
    
    # Summary using official ARC scoring
    total_tests = sum(r.n_test_cases for r in results)
    total_correct = sum(r.n_correct for r in results)
    tasks_fully_solved = sum(1 for r in results if r.score == 1.0)
    errors = sum(1 for r in results if r.error)
    avg_score = sum(r.score for r in results) / len(results) if results else 0
    
    print("\n" + "=" * 60)
    print("🏆 RETRY RESULTS (Official ARC Scoring)")
    print("=" * 60)
    print(f"📋 Tasks evaluated: {len(results)}")
    print(f"📋 Total test cases: {total_tests}")
    print(f"✅ Test cases correct: {total_correct}/{total_tests} ({100*total_correct/total_tests:.1f}%)")
    print(f"🎯 Tasks fully solved: {tasks_fully_solved}/{len(results)}")
    print(f"📊 Average task score: {avg_score*100:.1f}%")
    print(f"❌ Errors: {errors}")
    print(f"⏱️  Total time: {duration:.0f}s ({duration/len(results):.1f}s per task avg)")
    
    print("\n📋 Individual results:")
    for r in results:
        status = "✓" if r.score == 1.0 else "◐" if r.score > 0 else "✗"
        tests_info = f" ({r.n_correct}/{r.n_test_cases} tests)" if r.n_test_cases > 1 else ""
        print(f"   {status} {r.task_id}: {r.score*100:.0f}%{tests_info}")
    
    # Show which previously failed tasks now pass
    newly_passing = [r.task_id for r in results if r.score == 1.0]
    if newly_passing:
        print(f"\n🎉 Fully solved: {newly_passing}")
    
    return results

# Run it!
results = asyncio.get_event_loop().run_until_complete(run_failed_tasks())


✓ Found 83/83 valid tasks in fail_tasks.txt
🔄 RETRYING FAILED TASKS (5 tasks)
   Total failed: 83 tasks

📋 Selected: ['247ef758.json', 'de809cff.json', 'da515329.json', '221dfab4.json', 'edb79dae.json']...
🚀 Running 5 tasks (max 100 concurrent)

🚀 [1/5] Starting: 247ef758.json
   📋 Task has 2 test inputs
🎭 ARC SOLVER
   Training examples: 3
   Test inputs: 2
------------------------------------------------------------
  👁️ Perceiving grids...

🚀 [2/5] Starting: de809cff.json
🎭 ARC SOLVER
   Training examples: 2
   Test inputs: 1
------------------------------------------------------------
  👁️ Perceiving grids...

🚀 [3/5] Starting: da515329.json
🎭 ARC SOLVER
   Training examples: 3
   Test inputs: 1
------------------------------------------------------------
  👁️ Perceiving grids...

🚀 [4/5] Starting: 221dfab4.json
   📋 Task has 2 test inputs
🎭 ARC SOLVER
   Training examples: 2
   Test inputs: 2
------------------------------------------------------------
  👁️ Perceiving grids...

🚀 

KeyboardInterrupt: 

     [gemini-flash] Attempt 3/10: ✗ Failed training
     [gemini-flash] Attempt 4/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 5/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 6/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 7/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 8/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 9/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 10/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] ❌ Exhausted 10 tries
     [gemini-flash] Attempt 6/10: ✗ Failed training
     [gemini-flash] Attempt 7/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 
     [gemini-flash] Attempt 8/10: ✗ Error - Error code: 401 - {'error': {'message': 'User not 